In [1]:
import sqlite3
import pandas as pd
import numpy as np
from API_Key import key, username, password
import requests
import time
import psycopg2 as pg2
import json

In [2]:
def inputting_to_db(username, password, query):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute(query)
    
    conn.commit()
    conn.close()

In [3]:
# Adding data to tables created 
def adding_to_db(username, password, query, data):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.executemany(query, data)
    
    conn.commit()
    conn.close()

# Creating tables based on Riot Games API Structure

In [4]:
query1 = ('''
            CREATE TABLE summoner(

                leagueId VARCHAR,
                queueType VARCHAR,
                tier TEXT,
                rank TEXT,
                summonerId VARCHAR,
                summonerName TEXT,
                leaguepoints INTEGER,
                wins INTEGER,
                losses INTEGER,
                veteran BOOLEAN,
                inactive BOOLEAN,
                freshBlood BOOLEAN,
                hotStreak BOOLEAN) ''')

In [36]:
query2 = ('''  
                CREATE TABLE userids(

                id TEXT,
                accountId TEXT,
                puuid TEXT PRIMARY KEY,
                name TEXT,
                profileIconId INTEGER,
                revisionDate NUMERIC,
                summonerLevel INTEGER) ''')

In [14]:
query3 = (""" 
             CREATE TABLE matchids(
             puuid text,
             match text)""")

In [4]:
query4 = 'INSERT INTO summoner VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [5]:
query5 = 'INSERT INTO userids VALUES (%s,%s,%s,%s,%s,%s,%s)'

In [12]:
query6 = 'INSERT INTO matchids VALUES (%s,%s)'

In [25]:
query7 = (''' 
            CREATE TABLE matches(
                match TEXT,
                game_datetime NUMERIC,
                game_length FLOAT,
                game_version VARCHAR,
                gold_left INT,
                last_round INT,
                level INT,
                placement INT,
                puuid TEXT,
                time_eliminated FLOAT,
                total_damage INT,
                traits TEXT,
                units TEXT) ''')

In [10]:
query8 = 'INSERT INTO matches VALUES (%s,%s, %s,%s,%s,%s,%s, %s, %s, %s, %s, %s, %s)'

In [8]:
inputting_to_db(username, password, query1)

In [37]:
inputting_to_db(username, password, query2)

In [15]:
inputting_to_db(username, password, query3)

In [26]:
inputting_to_db(username, password, query7)

# Setting up loop to obtain all ranked TFT players in Diamond and below

In [12]:
def player_collector(key):
    leagues = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
    divisions = ['I', 'II', 'III', 'IV']
    summoner = []
    for league in leagues:
        for division in divisions:
            response = requests.get(
            "https://na1.api.riotgames.com/tft/league/v1/entries/{}/{}?api_key={}".format(league, division, key))
            for user in response.json():
                summoner.append(tuple(user.values()))
    return summoner

In [13]:
players = player_collector(key)

In [14]:
len(players)

4782

# Updating summoner table 

In [16]:
adding_to_db(username, password, query4, players)

In [18]:
def puuid_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT DISTINCT(summonerName) FROM summoner")
    players = c.fetchall()
    # Getting a list of all summonernames to loop through
    users = [player[0] for player in players]
    summoner = []
    for player in users:
        response = requests.get(
        "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{}?api_key={}".format(player, key))
        if response.status_code == 200:
            summoner.append(tuple(response.json().values()))
        elif response.status_code == 429:
            print(len(summoner))
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return summoner;

In [22]:
puuid = puuid_collector(key, username, password)

7
rate limit exceeded
waiting 2 minutes
105
rate limit exceeded
waiting 2 minutes
205
rate limit exceeded
waiting 2 minutes
305
rate limit exceeded
waiting 2 minutes
405
rate limit exceeded
waiting 2 minutes
505
rate limit exceeded
waiting 2 minutes
605
rate limit exceeded
waiting 2 minutes
704
rate limit exceeded
waiting 2 minutes
804
rate limit exceeded
waiting 2 minutes
904
rate limit exceeded
waiting 2 minutes
1004
rate limit exceeded
waiting 2 minutes
1104
rate limit exceeded
waiting 2 minutes
1204
rate limit exceeded
waiting 2 minutes
1304
rate limit exceeded
waiting 2 minutes
1404
rate limit exceeded
waiting 2 minutes
1504
rate limit exceeded
waiting 2 minutes
1603
rate limit exceeded
waiting 2 minutes
1703
rate limit exceeded
waiting 2 minutes
1803
rate limit exceeded
waiting 2 minutes
1903
rate limit exceeded
waiting 2 minutes
2003
rate limit exceeded
waiting 2 minutes
2102
rate limit exceeded
waiting 2 minutes
2202
rate limit exceeded
waiting 2 minutes
2302
rate limit exceede

In [38]:
adding_to_db(username, password, query5, puuid)

In [6]:
def matchid_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT puuid FROM userids")
    players = c.fetchall()
    # Getting a list of all puuids to loop through
    users = [player[0] for player in players]
    games = []
    for player in users:
        response = requests.get(
            "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{}/ids?count=30&api_key={}".format(player, key))
        if response.status_code == 200:
            for matches in response.json():
                games.append((player, matches))
        elif response.status_code == 429:
            print(len(games), games[-2:])
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return games;

In [10]:
match_histories = matchid_collector(key, username, password)

2918 [('T0o5tn_pyCXbcGn56CStM601xGZ56eBwwcyKm5p7bR07oMQsGjIAn7MW7cE3gZTcVlTYtuuSopvbKQ', 'NA1_4105749611'), ('T0o5tn_pyCXbcGn56CStM601xGZ56eBwwcyKm5p7bR07oMQsGjIAn7MW7cE3gZTcVlTYtuuSopvbKQ', 'NA1_4105698324')]
rate limit exceeded
waiting 2 minutes
5891 [('3bXpK0ak9JbOWeaKn06-u-9I9uU8NyWa3LeiBT1om87iao1ZUn_T6AkiuPR3yycNPgeqqBqW8qyPEQ', 'NA1_4092926462'), ('3bXpK0ak9JbOWeaKn06-u-9I9uU8NyWa3LeiBT1om87iao1ZUn_T6AkiuPR3yycNPgeqqBqW8qyPEQ', 'NA1_4092883517')]
rate limit exceeded
waiting 2 minutes
8814 [('ud420M--WdG7B1XNX4CCMB_uaLSJiyZyF6ksD_k6UlrvwFnREKFwK0gfPMXlnrdD78PQMxchYDfNsw', 'NA1_4100482847'), ('ud420M--WdG7B1XNX4CCMB_uaLSJiyZyF6ksD_k6UlrvwFnREKFwK0gfPMXlnrdD78PQMxchYDfNsw', 'NA1_4100137392')]
rate limit exceeded
waiting 2 minutes
11790 [('wknvNIrwzlGQJAj1Vb2tLqMOWBtPl0AGQvrVzkoh2KzaVdKeHNCmyZYg3zszhY1eKANUl7jEGCB4Qw', 'NA1_4105791283'), ('wknvNIrwzlGQJAj1Vb2tLqMOWBtPl0AGQvrVzkoh2KzaVdKeHNCmyZYg3zszhY1eKANUl7jEGCB4Qw', 'NA1_4105224516')]
rate limit exceeded
waiting 2 minutes
14719 [

100559 [('Zo0HQLO6hFvciFW68-ehGELUZPxoz5teapEZaJTJ9msAdH0edD6mhP4YLVlJdtsN9Y7wZWHejHBCFQ', 'NA1_4101189641'), ('Zo0HQLO6hFvciFW68-ehGELUZPxoz5teapEZaJTJ9msAdH0edD6mhP4YLVlJdtsN9Y7wZWHejHBCFQ', 'NA1_4101137375')]
rate limit exceeded
waiting 2 minutes
103528 [('O8xyJPpTpcbwTRkpfiwYmreRz4yhLRdHfFxP_s6ow07Z_B9kzre3GjTSfH94cTnsIktCVcZpw9agew', 'NA1_3201756389'), ('O8xyJPpTpcbwTRkpfiwYmreRz4yhLRdHfFxP_s6ow07Z_B9kzre3GjTSfH94cTnsIktCVcZpw9agew', 'NA1_3196993852')]
rate limit exceeded
waiting 2 minutes
106486 [('r7Mr8sU9D24KPikKfgR0CDxLqbLkq-jZgKQFHEt_CDqNim869OheSxBnbvdc2tIPXw-QvPsT7H46rg', 'NA1_4103576195'), ('r7Mr8sU9D24KPikKfgR0CDxLqbLkq-jZgKQFHEt_CDqNim869OheSxBnbvdc2tIPXw-QvPsT7H46rg', 'NA1_4103570681')]
rate limit exceeded
waiting 2 minutes
109440 [('DCYf8nFYTkvq-dkmTsvHVHJ_ZOx3iIyPoGkwW5xLRVo8wAgOHOZ1oHtOUSSSrymVjB6brIywt6V4bA', 'NA1_4060803241'), ('DCYf8nFYTkvq-dkmTsvHVHJ_ZOx3iIyPoGkwW5xLRVo8wAgOHOZ1oHtOUSSSrymVjB6brIywt6V4bA', 'NA1_4060518781')]
rate limit exceeded
waiting 2 minutes


In [16]:
adding_to_db(username, password, query6, match_histories)

In [19]:
match_data = []
for player in response.json()['info']['participants']:
                game_datetime = response.json()['info']['game_datetime']
                game_length = response.json()['info']['game_length']
                game_version = response.json()['info']['game_version']
                gold_left = player['gold_left']
                last_round = player['last_round']
                level = player['level']
                placement = player['placement']
                puuid = player['puuid']
                time_eliminated = player['time_eliminated']
                total_damage = player['total_damage_to_players']
                traits = player['traits']
                units = player['units']
                match_data.append(
                    (game_datetime, game_length, game_version, gold_left, last_round, level, placement,
                    puuid, time_eliminated, total_damage, traits, units))

In [5]:
# A lot of data so going to call the matchids in batches
def match_data_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT DISTINCT(match) FROM matchids WHERE match NOT IN (SELECT match FROM matches) LIMIT 1000")
    matches = c.fetchall()
    # Getting a list of all puuids to loop through
    games = [match[0] for match in matches]
    match_data = []
    for game in games:
        response = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/{}?api_key={}".format(game, key))
        start_time = time.time()
        if response.status_code == 200:
            for player in response.json()['info']['participants']:
                game_id = game
                game_datetime = response.json()['info']['game_datetime']
                game_length = response.json()['info']['game_length']
                game_version = response.json()['info']['game_version']
                gold_left = player['gold_left']
                last_round = player['last_round']
                level = player['level']
                placement = player['placement']
                puuid = player['puuid']
                time_eliminated = player['time_eliminated']
                total_damage = player['total_damage_to_players']
                traits = json.dumps(player['traits'])
                units = json.dumps(player['units'])
                match_data.append(
                    (game_id, game_datetime, game_length, game_version, gold_left, last_round, level, placement,
                    puuid, time_eliminated, total_damage, traits, units))
                end_time = time.time()
                
                #if end_time - start_time < 1:
                 #   time.sleep(1 - (end_time - start_time))
                
        elif response.status_code == 429:
            print(match_data[-1])
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return match_data;

In [7]:
conn = pg2.connect(database='TFT', user=username, password=password)
c = conn.cursor()
c.execute("SELECT DISTINCT(match) FROM matchids WHERE match NOT IN (SELECT match FROM matches)")
matches = c.fetchall()
# Getting a list of all puuids to loop through 

In [9]:
matches = match_data_collector(key, username, password)

('NA1_4080088612', 1635049301247, 2083.8935546875, 'Version 11.21.403.3002 (Oct 14 2021/15:48:27) [PUBLIC] <Releases/11.21>', 3, 30, 8, 5, 'JDEnUljgkFt2MXKa4lzFRRQUo-ovXXh0gK3TC9o3h516ARiOSX3xxjZfJahIxHFFDVXqukxOONOVyg', 1714.4530029296875, 84, '[{"name": "Set5_Abomination", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 3}, {"name": "Set5_Brawler", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 3}, {"name": "Set5_Caretaker", "num_units": 1, "style": 3, "tier_current": 1, "tier_total": 1}, {"name": "Set5_Dawnbringer", "num_units": 4, "style": 2, "tier_current": 2, "tier_total": 4}, {"name": "Set5_Draconic", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 2}, {"name": "Set5_Invoker", "num_units": 2, "style": 1, "tier_current": 1, "tier_total": 2}, {"name": "Set5_Mystic", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 4}, {"name": "Set5_Renewer", "num_units": 6, "style": 4, "tier_current": 3, "tier_total": 3}, {"name": "Set5_Revenant

('NA1_3801113065', 1614180282884, 2107.660888671875, 'Version 11.4.360.0513 (Feb 18 2021/13:57:42) [PUBLIC] <Releases/11.4>', 21, 31, 9, 5, '46qaFJWY3mhldh3wrpC-ZMSrmm51m6zcn6PxR96G4ktCWGwYiOy8vx29m8xPzmJucklsZ3JXAu14TQ', 1752.1162109375, 95, '[{"name": "Boss", "num_units": 1, "style": 3, "tier_current": 1, "tier_total": 1}, {"name": "Divine", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 4}, {"name": "Duelist", "num_units": 3, "style": 1, "tier_current": 1, "tier_total": 4}, {"name": "Emperor", "num_units": 1, "style": 3, "tier_current": 1, "tier_total": 1}, {"name": "Fortune", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 2}, {"name": "Keeper", "num_units": 2, "style": 1, "tier_current": 1, "tier_total": 3}, {"name": "Set4_Assassin", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 3}, {"name": "Set4_Brawler", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 4}, {"name": "Set4_Daredevil", "num_units": 1, "style": 3, "tier_current"

In [28]:
len(matches)

7928

In [27]:
adding_to_db(username, password, query8, matches)